In [4]:
import torch
import torchaudio
import torchaudio.sox_effects as ta_sox

from transformers import AutoModelForCTC, Wav2Vec2Processor

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = AutoModelForCTC.from_pretrained("bhuang/asr-wav2vec2-french").to(device)
processor = Wav2Vec2Processor.from_pretrained("bhuang/asr-wav2vec2-french")
model_sample_rate = processor.feature_extractor.sampling_rate

In [11]:
audio_path = "record-1.wav"  # path to your audio file
waveform, sample_rate = torchaudio.load(audio_path)

effects = []
if model_sample_rate != sample_rate:
    # resample
    effects.append(["rate", f"{model_sample_rate}"])
if waveform.shape[0] > 1:
    # convert to mono
    effects.append(["channels", "1"])
if len(effects) > 0:
    converted_waveform, _ = ta_sox.apply_effects_tensor(waveform, sample_rate, effects)

# 1d array
converted_waveform = converted_waveform.squeeze(axis=0)

# normalize
input_dict = processor(converted_waveform, sampling_rate=model_sample_rate, return_tensors="pt")

with torch.inference_mode():
    # forward
    logits = model(input_dict.input_values.to(device)).logits

# decode
predicted_ids = torch.argmax(logits, dim=-1)
predicted_sentence = processor.batch_decode(predicted_ids)[0]
predicted_sentence

"bonjour chorgepet j'aimerais que tu m'écrives une preuve mathématique comme quoi la moyenne d'un ensemble de valeurs sera toujours plus grande ou égale à la plus petite valeur dans cet ensemble de valeurs"